In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from gensim.utils import simple_preprocess
from gensim.models import LsiModel, LdaModel
from gensim import corpora
import spacy

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = stopwords.words('english')
sp = spacy.load('en_core_web_lg')

categories = ['rec.autos', 'comp.graphics', 'sci.space']
newsgroup = fetch_20newsgroups(categories=categories, shuffle=True)
data = newsgroup.data
target = newsgroup.target

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def lemmatizeNLTK(input):
    lemmatizer = WordNetLemmatizer()
    #Tokenizers divide strings into lists of substrings
    wordList = word_tokenize(input)
    output = ' '.join([lemmatizer.lemmatize(w) for w in wordList])
    return output

In [3]:
def lemmatizeSpacy(input):
    doc = sp(input)
    #Tokenizers divide strings into lists of substrings
    output = ' '.join([w.lemma_ for w in doc])
    return output

In [4]:
def NExt(input):
    output=""
    doc = sp(input)
    for w in doc:
        if w.pos_=="NOUN":
            output+=w.text+" "
    output = output[:-1]
    return output

In [5]:
def NAExt(input):
    output=""
    doc = sp(input)
    for w in doc:
        if w.pos_ in ["NOUN","ADJ"]:
            output+=w.text+" "
    output = output[:-1]
    return output

In [6]:
def NAVExt(input):
    output=""
    doc = sp(input)
    for w in doc:
        if w.pos_ in ["NOUN","ADJ","VERB"]:
            output+=w.text+" "
    output = output[:-1]
    return output

In [7]:
def NVExt(input):
    output=""
    doc = sp(input)
    for w in doc:
        if w.pos_ in ["NOUN","VERB"]:
            output+=w.text+" "
    output = output[:-1]
    return output

In [8]:
def train(x_train, x_test, y_train, y_test, clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    return f1_score(y_test, y_pred, average='weighted')

In [9]:
def textProcessing(unprocessedTexts, function=None):
    if function is None:
        return unprocessedTexts
    return [function(text) for text in unprocessedTexts]

In [ ]:
mapPreProc = ["None","Noun","Noun+Adj","Noun+Adj+Verb","Noun+Verb"]
mapProc = ["None", "LemNLTK", "LemSpacy"]
mapVec = ["BoW","TF-IDF","LSI","LDA"]
mapAct = ['identity', 'logistic', 'tanh', 'relu']
mapSol = ['lbfgs', 'sgd', 'adam']
mapIter = [100,125,150,200,250,300,400,500]
mapRS = [1,10,15,25,40]
list=[]
for i in range(5):
    if i==0:
        preProc=None
    if i==1:
        preProc=NExt
    if i==2:
        preProc=NAExt
    if i==3:
        preProc=NAVExt
    if i==4:
        preProc=NVExt
    dataExt = textProcessing(data,preProc)
    for j in range(3):
        if j==0:
            proc=None
        if j==1:
            proc=lemmatizeNLTK
        if j==2:
            proc=lemmatizeSpacy
        dataProc = textProcessing(data,proc)
        tokenized_documents = [simple_preprocess(text) for text in dataProc]
        dictionary = corpora.Dictionary(tokenized_documents)
        bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]
        lsi = LsiModel(bow_corpus,id2word=dictionary, num_topics=10)
        lda = LdaModel(bow_corpus, num_topics=25, id2word=dictionary,passes=15, minimum_probability = 0)
        for k in range(4):
            if k==0:
                vect = CountVectorizer(binary=False, stop_words=stop_words)
            if k==1:
                vect = TfidfVectorizer(stop_words=stop_words)
            if k==2:
                func = lsi
            if k==3:
                func = lda
            X_train, X_test, y_train, y_test = [],[],[],[]
            if k in [0,1]:
                trainProc, testProc, y_train, y_test = train_test_split(dataProc, target, test_size=0.33)
                X_train = vect.fit_transform(trainProc)
                X_test = vect.transform(testProc)
            else:
                dataVecPre = func[bow_corpus]
                len_data = len(dataVecPre)
                dataVec = [0 for o in range(len_data)]
                for o in range(len_data):
                    curr = []
                    a = dataVecPre[o]
                    length = len(a)
                    curr = [a[u][1] for u in range(length)]
                    dataVec[o] = curr
                X_train, X_test, y_train, y_test = train_test_split(dataVec, target, test_size=0.33)
            for m_i in mapIter:
                for act in mapAct:
                    for sol in mapSol:
                        for rs in mapRS:
                            clf = MLPClassifier(max_iter=m_i, activation=act, solver=sol, random_state=rs)
                            f1 = train(X_train, X_test, y_train, y_test, clf)
                            s = " ".join([mapPreProc[i],mapProc[j],mapVec[k],str(m_i), act, sol, str(rs)])
                            list.append([f1,s])
                            print(s+": "+str(f1))
        print("-------------------------------------------")
    print("-------------------------------------------")
listMax = [el[0] for el in list]
maximum = max(listMax)
print(f"Maximum: {maximum}")
print(list[listMax.index(maximum)][1])

None None BoW 100 identity lbfgs 1: 0.9607670868596317
None None BoW 100 identity lbfgs 10: 0.9641684552805725
None None BoW 100 identity lbfgs 15: 0.9607714472002739
None None BoW 100 identity lbfgs 25: 0.9660294151966398
None None BoW 100 identity lbfgs 40: 0.9573924650847729


C:\Users\zhest\PycharmProjects\LSINewsGroup\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


None None BoW 100 identity sgd 1: 0.9506568192277054


C:\Users\zhest\PycharmProjects\LSINewsGroup\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


None None BoW 100 identity sgd 10: 0.9523125970546967


...

None None TF-IDF 400 relu sgd 15: 0.937941587264205


C:\Users\zhest\PycharmProjects\LSINewsGroup\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


None None TF-IDF 400 relu sgd 25: 0.9276188476921247


C:\Users\zhest\PycharmProjects\LSINewsGroup\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


None None TF-IDF 400 relu sgd 40: 0.9325969902114819
None None TF-IDF 400 relu adam 1: 0.9743922879408881
None None TF-IDF 400 relu adam 10: 0.9777845736936478
None None TF-IDF 400 relu adam 15: 0.9761045827207644
None None TF-IDF 400 relu adam 25: 0.9692869777435228
None None TF-IDF 400 relu adam 40: 0.9743905508528482
None None TF-IDF 500 identity lbfgs 1: 0.9778406902215796
None None TF-IDF 500 identity lbfgs 10: 0.9761057693679733
None None TF-IDF 500 identity lbfgs 15: 0.9726750327654228
None None TF-IDF 500 identity lbfgs 25: 0.974397763011645
None None TF-IDF 500 identity lbfgs 40: 0.9726826030570608
